In [102]:
import numpy as np
import csv
import pandas as pd
from sklearn.cross_validation import train_test_split
import math

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'

train_size = .8
K = 40
learn_rate = 0.0000001
iterations = 10
w = .8


#load CSV
train_data = {}
test_data = {}
ndata = 0
with open(train_file, 'r') as train_fh:
    ndata = sum(1 for row in train_fh) - 1
    
splitting = np.random.binomial(1, train_size, size=ndata)
print ndata*train_size


 3323843.2


In [103]:

with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    
    for i, row, in enumerate(train_csv):
        user   = row[0]
        artist = row[1]
        plays  = row[2]

        if splitting[i]: 
            if not user in train_data:
                train_data[user] = {}
            train_data[user][artist] = int(plays)
            
        else:
            if not user in test_data:
                test_data[user] = {}

            test_data[user][artist] = int(plays)
              
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))
print global_median

118.0


In [104]:
user_feats = {}
artist_feats = {}


init = np.array(K * [math.sqrt(float(global_median)/K)])


artist_plays = {}
for user, user_data in train_data.iteritems():
    if not user in user_feats:
        user_feats[user] = init
    for artist, plays in user_data.iteritems():
        if not artist in artist_plays:
            artist_plays[artist] = []
        artist_plays[artist].append(plays)
        if not artist in artist_feats:
            artist_feats[artist] = init
 

[ 1.7175564  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564
  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564
  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564
  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564
  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564
  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564  1.7175564
  1.7175564  1.7175564  1.7175564  1.7175564]


In [105]:
x = artist_feats.keys()[1]

y = artist_plays[x]

            
artist_meds = {}
for artist in artist_plays.keys():
    artist_meds[artist] = np.median(np.array(artist_plays[artist]))

 

In [106]:
i = 7
test_artist = artist_feats.keys()[i]

print artist_meds[test_artist]


218.5


In [107]:
def predict_plays(user, artist):
    if (not user in user_feats) or (not artist in artist_feats):
        return global_median
    else:
        return np.dot(np.array(user_feats[user]), np.array(artist_feats[artist]))

    
def modify_prediction(user, artist, prediction):
    user_bias = global_median - user_medians[user]
    
    return w*prediction + (1-w)*(artist_meds[artist] + user_bias)

def train(data):
    i = 0
    
    for k in range(iterations):
        for j, (user, user_data) in enumerate(train_data.iteritems()):
            if j % 1000 == 0:
                print "\r", j,k,

            for artist, plays in user_data.iteritems(): 
                
                pred = predict_plays(user,artist)
                error = int(plays) - pred

                artist_feats[artist] += learn_rate*error*user_feats[user]
                user_feats[user] +=  learn_rate*error*artist_feats[artist]  

train(train_data)

233000 9


In [108]:
def dumb_check(data):
    errors = []
    pred = 0
    for j, (user, user_data) in enumerate(data.iteritems()):
        if j % 1000 == 0:
            print "\r", j,
        for artist, plays in user_data.iteritems():
                pred = predict_plays(user,artist)
                
                newpred = modify_prediction(user,artist,pred)
            
      
                error = abs(int(plays) - newpred)
                
                errors.append(error)
               
                
    return sum(np.array(errors))/len(errors)
                
print dumb_check(test_data)




226000 239.090919168
